## Hartree Fock para el estado fundamental del Helio

In [1]:
#Librerias matematicas
import numpy as np

#Coso para resolver el problema de autovalores generalizado (eigvalsh devuelve solo autovalores)
from scipy.linalg import  eigh , eigvalsh

#Librerias para manipular cosas de forma simbolica
import sympy as sp

#Cosas para plotear
%matplotlib inline
import matplotlib as plt

In [2]:
z = 1 #Helio

nsize = 4
S = np.zeros((nsize,nsize))  # Overlap
H = np.zeros((nsize,nsize))  # One-electron Hamiltonian
G = np.zeros((nsize,nsize))  # Two-electron Operator
Q = np.zeros((nsize,nsize))
F = np.zeros((nsize,nsize))  # Fock Operator Hij + Gij

In [3]:
# Gaussian Basis  Ci*exp(-Alpha*r^2)
Alpha = np.zeros((nsize))    


# Datos Iniciales
Alpha[0] = 0.298073
Alpha[1] = 1.242567
Alpha[2] = 5.782948
Alpha[3] = 38.47497
#Alpha[4] = 0.1
#Alpha[5] = 5
#Alpha[6] = 20.
#Alpha[7] = 50.0

In [4]:
# Cálculo de Overlaps  Sij=<xi|xj>

def overlap(Alpha,p,q):
    spq = ( np.pi / (Alpha[p] + Alpha[q]) )**(3./2.)
    
    return spq

for i in range(0,nsize):
    for j in range(i,nsize):        
        S[i,j]=overlap(Alpha,i,j)
        S[j,i]=S[i,j]


In [5]:
 # Cálculo de Hamiltoniano Hij=<xi| H |xj>


# Cálculo de Energía Cinética  Tij=<xi|-1/2 D^2 |xj>
def Tkin(Alpha,p,q):
    rnum = 3 * np.pi**(3./2.) * Alpha[p] * Alpha[q]
    rden =  (Alpha[p] + Alpha[q] )**(5./2.)    
    Tpq = rnum/rden    
    return Tpq


# Cálculo de Energía Potencial  Vij=<xi|-Z/r |xj>
def Vpot(Alpha,p,q):
    Vpq = z * ( -2*np.pi )  /  (Alpha[p] + Alpha[q])    
    return Vpq

for i in range(0,nsize):
    for j in range(i,nsize): 
        H[i,j]=Tkin(Alpha,i,j) + Vpot(Alpha,i,j)
        H[j,i]=H[i,j]
   

In [6]:
# Cálculo de Repulsión Interelectrónica 
# Qirjs=<xi(r1)xr(r2)| 1/|r1-r2| |xj(r1)xs(r2)>


# Cálculo de Integral de Repulsión
# Qirjs=<xi(r1)xr(r2)| 1/|r1-r2| |xj(r1)xs(r2)>
# Ecuación (4.17) del Thijssen

def Qirjs(Alpha,i,r,j,s):
    rnum = 2 * np.pi**(5./2.) 
    sqrsum = np.sqrt(Alpha[i]+Alpha[j]+Alpha[r]+Alpha[s])
    rden =  (Alpha[i]+Alpha[j])*(Alpha[r]+Alpha[s])*sqrsum    
    Qirjs = rnum/rden    
    return Qirjs



In [7]:
#Aca voy a iterar Hartree Fock 15 veces

Ci = np.zeros((nsize)) #Inicializo los coeficientes con ceros

EnergyGS = []

for m in range(0,15):

    #Me armo el operador Q
    for i in range(0,nsize):
        for j in range(i,nsize):
            sum = 0
            for r in range(0,nsize):
                for s in range(0,nsize):
                    sum = sum + Qirjs(Alpha,i,r,j,s)*Ci[r]*Ci[s]
            Q[i,j] = sum
            Q[j,i] = sum

    #Armo el operador de Fock
    F = H + Q

    #Ener = eigvalsh(F,S,type=1)
    Ener,coef = eigh(F,S,type=1) #Me tira 4 autoenergias y 4 autovectores

    Ci = coef[:,0]

    #Ener[0],coef[:,0]

    #Normalizo los coeficientes C

    sum = 0

    for i in range(0,nsize):
        for j in range(0,nsize):
            sum = sum + Ci[i]*S[i,j]*Ci[j]

    Ci = Ci/np.sqrt(sum)

    # Energía Total


    sum = 0.0
    for i in range(0,nsize):
        for j in range(0,nsize):
            sum = sum + Ci[i]*Ci[j]*(2.0*H[i,j]+Q[i,j])

    EnergyGSnew = sum
    #EnergyGSnew

    EnergyGS.append(EnergyGSnew)
    
EnergyGS

[-0.9607965036865096,
 -0.2471261265106073,
 -0.2879556327498642,
 -0.280514839664046,
 -0.2818838235736279,
 -0.2816320283692169,
 -0.28167834748399456,
 -0.2816698270103204,
 -0.28167139437150174,
 -0.28167110605194534,
 -0.2816711590889731,
 -0.2816711493326932,
 -0.28167115112738317,
 -0.28167115079724575,
 -0.28167115085797517]

In [9]:
(11/16)**2

0.47265625